In [6]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [3]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        
    
def do_bot_things(data,time_to_sleep):
    while(True):
        
        # 1 get infos
        
        # 2 create inputs
        
        # 3 go through graph
        
        #https://stackoverflow.com/questions/37891954/keras-how-do-i-predict-after-i-trained-a-model
        #prediction = model.predict(np.array(tk.texts_to_sequences(text)))
        #print(prediction)
        
        # 4 depending on output : buy, sell or do nothing
        
        time.sleep(time_to_sleep)

In [4]:
stream = SwissQuoteStream()

In [5]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 10

window = []
previous_ema = 0
def handle_new_price(data):
    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!
    
    #print(str(new_price) + ' ' + str(ema))
    #print(str(diff))
    
    if(diff > 0):
        stream.sell(0.1)
        print('SELL!! Diff is: ' + str(diff))
    else:
        stream.buy(0.1)
        print('BUY!!! Diff is: ' + str(diff))

        


In [ ]:
stream.start(handle_new_price)

0.0
BUY!!! Diff is: 0.0
0.007310585786399315
SELL!! Diff is: 0.007310585786399315
0.005064803910954652
SELL!! Diff is: 0.005064803910954652
0.0038493697420562967
SELL!! Diff is: 0.0038493697420562967
0.003100220098531281
SELL!! Diff is: 0.003100220098531281
0.0025939860543076065
SELL!! Diff is: 0.0025939860543076065
0.002229436448942579
SELL!! Diff is: 0.002229436448942579
-0.007326431696128566
BUY!!! Diff is: -0.007326431696128566
0.0024651852249917283
SELL!! Diff is: 0.0024651852249917283
0.0022879041803207656
SELL!! Diff is: 0.0022879041803207656
0.000804870760475751
SELL!! Diff is: 0.000804870760475751
0.0008994584231913905
SELL!! Diff is: 0.0008994584231913905
0.0010051619392470457
SELL!! Diff is: 0.0010051619392470457
0.001123287634072767
SELL!! Diff is: 0.001123287634072767
0.001255295351256791
SELL!! Diff is: 0.001255295351256791
0.0014028164914634544
SELL!! Diff is: 0.0014028164914634544
-0.007855610720980621
BUY!!! Diff is: -0.007855610720980621
0.0006444901223403576
SELL!! D

BUY!!! Diff is: -0.06046318879816681
-0.055305425922142604
BUY!!! Diff is: -0.055305425922142604
-0.050205283513605536
